In [1]:

import pandas as pd
import torch
from transformers import BertTokenizer


c:\Users\User\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.12.1


In [3]:
model = torch.load('emo1017.pt', map_location='cpu')

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [36]:
stri = '牛肉漢堡居然可以做得這麼好吃'
inputs = tokenizer(stri, return_tensors="pt")
print(inputs)
#labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# tokens_tensors, segments_tensors, masks_tensors = inputs[:3]
outputs = model(input_ids=inputs['input_ids'], token_type_ids=inputs['token_type_ids'], 
                            attention_mask=inputs['attention_mask'])
logits = outputs[0]
_, cat =  torch.max(logits.data, 1)
cat = cat.detach().numpy()
pos = float(logits[0][1])
neg = float(logits[0][0])
score =  (( pos - neg )*1.25 + 10) / 4
print('pos:' , pos)
print('neg:' , neg)

print(round(score, 4))


{'input_ids': tensor([[ 101, 4281, 5489, 4031, 1836, 2233, 4197, 1377,  809,  976, 2533, 6857,
         7938, 1962, 1391,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
pos: 1.970383644104004
neg: -1.1603922843933105
3.4784


In [22]:
df = pd.read_csv("test.tsv", sep='\t')
df['pred'] = 0
df

,comment,stars,pred
0,是老店。它的沾醬 是用蝦殼、頭熬煮的 所以非常好吃。還有賣四神湯&芋稞。,1,0
1,空間明亮乾淨，服務人員態度親切。二樓座位區的長桌設有插座，使用三C充電方便。,1,0
2,餐點好吃，服務好！\n推薦西班牙油蝦，點了不會後悔。,1,0
3,用心的店家，餐點也好吃😋\n啤酒真的好喝還有伊比利豬肉跟蛋糕棒棒的👍,1,0
4,不知道是買不起夾子還是怎樣直接用手抓肉雖然有帶手套但不知道有沒有去做其他事情觀感真的很差有夠...,0,0
...,...,...,...
1437,「蝦仁飯+煎鴨蛋」\n蝦仁🍤有大火炒過特有的香味！\n飯的部分偏濕，若能有蝦味會更好\n煎鴨...,1,0
1438,用餐人潮不少，服務態度略差，不過雞肉飯味道算不錯，滷大腸也夠味。千萬不能帶外食，會被靠邀。,0,0
1439,週六晚上九點人不多，整體環境舒適。,1,0
1440,Cp不高，但員工服務不錯，外帶食物還會提供碗，不會不給食用。,0,0


In [25]:
for i in range(0, 1441):
    stri = df['comment'][i]
    inputs = tokenizer(stri, return_tensors="pt")
    #labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
    # tokens_tensors, segments_tensors, masks_tensors = inputs[:3]
    outputs = model(input_ids=inputs['input_ids'], token_type_ids=inputs['token_type_ids'], 
                                attention_mask=inputs['attention_mask'])
    logits = outputs[0]
    _, cat =  torch.max(logits.data, 1)
    cat = cat.detach().numpy()
    df['pred'][i] = int(cat)

C:\Users\User\AppData\Local\Temp\ipykernel_1088\412727776.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred'][i] = int(cat)


In [19]:
type(int(cat))

int

In [26]:
df

,comment,stars,pred
0,是老店。它的沾醬 是用蝦殼、頭熬煮的 所以非常好吃。還有賣四神湯&芋稞。,1,1
1,空間明亮乾淨，服務人員態度親切。二樓座位區的長桌設有插座，使用三C充電方便。,1,1
2,餐點好吃，服務好！\n推薦西班牙油蝦，點了不會後悔。,1,1
3,用心的店家，餐點也好吃😋\n啤酒真的好喝還有伊比利豬肉跟蛋糕棒棒的👍,1,1
4,不知道是買不起夾子還是怎樣直接用手抓肉雖然有帶手套但不知道有沒有去做其他事情觀感真的很差有夠...,0,0
...,...,...,...
1437,「蝦仁飯+煎鴨蛋」\n蝦仁🍤有大火炒過特有的香味！\n飯的部分偏濕，若能有蝦味會更好\n煎鴨...,1,1
1438,用餐人潮不少，服務態度略差，不過雞肉飯味道算不錯，滷大腸也夠味。千萬不能帶外食，會被靠邀。,0,0
1439,週六晚上九點人不多，整體環境舒適。,1,1
1440,Cp不高，但員工服務不錯，外帶食物還會提供碗，不會不給食用。,0,0


In [27]:
from sklearn.metrics import confusion_matrix
y_true = df['stars']
y_pred = df['pred']

confusion_matrix(y_true, y_pred)

array([[568,   1],
       [  6, 867]], dtype=int64)

In [28]:
from sklearn.metrics import classification_report


print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       569
           1       1.00      0.99      1.00       873

    accuracy                           1.00      1442
   macro avg       0.99      1.00      0.99      1442
weighted avg       1.00      1.00      1.00      1442

